In [4]:
import sys
#import cPickle
from importlib import reload

import numpy as np
import scipy as sp
import pandas as pd
import os
import itertools
import optimization as OPT
import MF
import utilities as UT
from matplotlib import rc
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
import seaborn as sns

reload(MF)
reload(OPT)
reload(UT)

Data_path = 'Data/'

algorithm = 'gradient_descent'

#Read Movielens Dataset
n_users=  300
n_movies= 500
top_users = False

X, genres, user_info = MF.read_movielens_1M(n_movies, n_users, top_users, data_dir = Data_path+'MovieLens-1M')
omega = ~X.isnull()

#print(user_info.columns)
num_rows = len(user_info)
user_ids = np.arange(1, num_rows + 1)

# Add a new 'UserID' column to the DataFrame
#user_ids = user_info.index +1
user_info['UserID'] = user_ids
#print(user_info)

G = {
    'M': [],  # Group for Male users
    'F': []   # Group for Female users
}

# Build the groups based on gender
for _, row in user_info.iterrows():
    user_id = row['UserID']
    gender = row['Gender']
    G[gender].append(user_id)


print(len(user_info))
print(len(user_ids))
print(user_info.head())

rank = 4
lambda_ = 0.1
axis = 0
RS = MF.als_MF(rank,lambda_) # Use the matrix factorization algorithm with alternating least sqaure as the optimization algorithm
utility = UT.individual_loss_variance(X,omega,axis)#group_loss_variance(X,omega,G,axis)

pred,error = RS.fit_model(X)
X_est = RS.pred.copy() #X_est is the predicted rating matrix
RMSE_before = MF.compute_RMSE(X,X_est,omega)

print (" Individual Fairness before :", utility.evaluate(X_est))
print("RMSE before : ",RMSE_before)
n,d = X.shape
budget_perc = 1.0
budget = int(n_users*budget_perc/100.0)
print ('budget:',budget)

#------------------------find optimal antidote--------------------------
runs = 3
stepsize = 1.0 #this only determines the direction if the gradient step. Positive values will minimize the objective and negative values will maximize it
projection = OPT.projection((0,5))
max_iter = 20
threshold = 0.1
window  = 1
steps = OPT.LineSearch_steps(10**3,6) #if None, the steps will be selected automatically based on the magnitude of the values in the gradient matrix
initial_data = 'random'

alg = OPT.gradient_descent_LS(max_iter, stepsize, threshold, steps, window)
results = alg.run(RS,X,budget,projection,utility,initial_data,runs)
X_antidote = results['X_antidote']
obj_history = results['obj_hist']
#-----------------------------------------------------------------------

#----------------------apply antidote data------------------------------
U_final,V_final,X_final = MF.antidote_effect(RS,X,X_antidote)
obj_after = utility.evaluate(X_final)
RMSE_after = MF.compute_RMSE(X,X_final,omega)

print("polarization after antidote data:",obj_after)
print("RMSE after antidote data: ",RMSE_after)
#-----------------------------------------------------------------------



6040
6040
  Gender  Age  Occupation Zip-code  UserID
1      F    1          10    48067       1
2      M   56          16    70072       2
3      M   25          15    55117       3
4      M   45           7    02460       4
5      M   25          20    55455       5
 Individual Fairness before : 0.03114815857018162
RMSE before :  0.7566184426794337
budget: 3
Run 1
obj_new:  0.03990822616933822
steps [1000.0, 100.0, 10.0, 1.0, 0.1, 0.01]
optimal step: 1000.0
obj_new:  0.0272192215819256
optimal step: 1000.0
obj_new:  0.025018159729909995
optimal step: 1000.0
obj_new:  0.02407260429516782
optimal step: 0.1
obj_new:  0.023657304270972717
optimal step: 1.0
obj_new:  0.023378701515708988
Run 2
obj_new:  0.03497569449748515
steps [1000.0, 100.0, 10.0, 1.0, 0.1, 0.01]
optimal step: 1000.0
obj_new:  0.02784947648356279
optimal step: 1000.0
obj_new:  0.024838855768861277
optimal step: 1000.0
obj_new:  0.02392586306778129
optimal step: 100.0
obj_new:  0.02325683021489337
Run 3
obj_new:  0.03696

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import timeit
import MF
reload(MF)

#Read Movielens Dataset
n_users=  1000
n_movies= 1000
top_users= True
X, genres, user_info = MF.read_movielens_1M(n_movies, n_users, top_users)

print(X.shape)
known = X.count().sum() / (1.0*X.size)
print(known)

n_known_item = (~X.isnull()).sum(axis=0).sort_values()
n_known_user = (~X.isnull()).sum(axis=1).sort_values()

#Factorization parameters
rank = 20
lambda_ = 20 #Ridge regularizer parameter

#Initiate a recommender system of type ALS
#RS = MF.als_MF(rank,lambda_)

#Initiate a recommender system of type lmafit
RS = MF.lmafit_MF(rank)

start = timeit.default_timer()
pred,error = RS.fit_model(X)
time = timeit.default_timer() - start
print ('factorization time:',time)
print ('RMSE:',np.sqrt(error/X.count().sum()))

V = RS.get_V()
U = RS.get_U()
print(U.shape)
print(V.shape)

(1000, 1000)
0.361426
factorization time: 8.479209823999554
RMSE: 0.7019747777066776
(1000, 20)
(20, 1000)
